In [ ]:
from openai import OpenAI
import json
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv()) # read local .env file

In [ ]:
client : OpenAI = OpenAI()

### Step 0: Define functions
First, define your functions:

In [ ]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def getCurrentWeather(location:str, unit:str="fahrenheit")->str | dict | None:
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "los angeles" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})
    

def getNickname(location:str)->str:
    """Get the nickname of a city"""
    if "tokyo" in location.lower():
        return "tk"
    elif "los angeles" in location.lower():
        return "la"
    elif "paris" in location.lower():
        return "py"
    else:
        return location



### Step 1: Create an Assistant and register/report your functions

In [ ]:
import json 
def show_json(message.obj):
      display(mesaage, json.load(obj.model_dump_json()))

In [ ]:
from openai.types.beta import Assistant

assistant: Assistant = client.beta.assistants.create(
  instructions="You are a weather bot. Use the provided functions to answer questions.",
  model="gpt-3.5-turbo-1106",
  tools=[{
      "type": "function",
    "function": {
      "name": "getCurrentWeather",
      "description": "Get the weather in location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
          "unit": {"type": "string", "enum": ["c", "f"]}
        },
        "required": ["location"]
      }
    }
  }, {
    "type": "function",
    "function": {
      "name": "getNickname",
      "description": "Get the nickname of a city",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
        },
        "required": ["location"]
      }
    } 
  }]
)


In [ ]:
from openai.types.beta.thread import Thread

thread: Thread  = client.beta.threads.create()

print(thread)


In [ ]:
from openai.types.beta.threads.thread_message import ThreadMessage

# First Request
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="How is the weather in Los Angles?"
)


In [ ]:
from openai.types.beta.threads.run import Run

run: Run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)


In [ ]:
available_functions = {
    "getCurrentWeather": getCurrentWeather,
    "getNickname": getNickname
} 

In [ ]:
import time 
while True:
    runstatus =client.beta.threads.run.retrieve(
        thread_id=thread.id
        run_id=run.id
    )
    run_steps=client.beta.threads.runs.steps.list(  thread_id=thread.id
        run_id=run.id)
    if runstatus.status== 'requires_actions':
        print(runstatus.status)
        print('Status': 'requires_action')
        if runstatus.required_action.submit_tool_outputs and runstatus.required_action.submit_tool_outputs.tools_calls:
                       toolCalls = runStatus.required_action.submit_tool_outputs.tool_calls
                       tool_ouput=[]
    for toolcall in toolcalls
    function_name=toolcall.function.name
    dunction_args=json.loads(toolcall.function.argumrnts)
    if (function_name in available_functions)
    function_To_call= available_functions[function_name]
    if (function_To_call.__name__)=='getcurrentweather'
    response= function_To_call(
        location=function_args.get('loction')
        unit=dunction_args.get('unt')
    )
    tool_ouput.append({
                                  "tool_call_id": toolcall.id,
                                  "output": response
                              })
    
    
    
      elif function_to_call.__name__ == "getNickname":
                        response = function_to_call(
                          location=function_args.get("location")
                          )
                        tool_outputs.append({
                          "tool_call_id": toolcall.id,
                          "output": response,
                              })
            print(tool_outputs,">>>>>")
            
            
            
            
            
            